# Implementation of Bloom Filter

Library

In [31]:
# !pip install bitarray
# !pip install pyodbc
# !pip install hashlib

import math
import hashlib
from bitarray import bitarray

#using bitarray so there's no chance value will be neither 0 or 1

Class: __Bloom Filter__
> Consist:
- Function: __init__: to initialize the class, functions, and class methods functions used
- Function: __add__: to insert a specified desired element to the array
- Function: __check__: to an existence of a specified element in the array
- Function: (classmethod) __get_size__: to calculate the size of the array
- Function: (classmethod) __get_hash_count__: to calculate the hash count needed to hash a specified element

In [32]:
class BloomFilter(object):

	'''
	Class for Bloom filter, using SHA256 hash function
	'''

	def __init__(self, items_count, fp_prob):
		'''
    Bloom Filter is a bit of array of specified size (m) and initially sets to zero

    Glosarium:
		  n = items_count : int
			  Number of items expected to be stored in bloom filter
		  p = fp_prob : float
			  False Positive probability in decimal
      k = hash count
        Hash count needed for specified value. Formula commented alongside the function.
      m = size of array
        m CAN'T BE INPUTED MANUALLY without calculating the items count and hash count. Otherwise, collision increases.
		'''

		# Initialize false positive probability in decimal
		self.fp_prob = fp_prob

		# Initialize size (m) of bit array to use
		self.size = self.get_size(items_count, fp_prob)

		# Initialize number of hash functions (k) to use
		self.hash_count = self.get_hash_count(self.size, items_count)

		# Initialize bit array of given size
    # Creating the array that will use the bloom filter method
		self.bit_array = bitarray(self.size)

		# Initialize all bits as 0
		self.bit_array.setall(0)

	def add(self, item):
		'''
		Encode and insert an item into the filter
		'''
		digests = []
		for i in range(self.hash_count):

			# create digest for given item.
			# using SHA256
      # checking the bit value
      # set bit value = position mod m
			digest = int(hashlib.sha256(item.encode()).hexdigest(),16) % self.size
			digests.append(digest)

			# set the bit True in bit_array
			self.bit_array[digest] = True

	def check(self, item):
		'''
		Check for existence of an item in filter
		'''
		for i in range(self.hash_count):
			digest = int(hashlib.sha256(item.encode()).hexdigest(),16) % self.size
			if self.bit_array[digest] == False:

				# if any of bit is False then,its not present
				# in filter
				# else there is probability that it exist
				return False
		return True

	@classmethod
	def get_size(self, n, p):
		'''
		Return the size of bit array(m) to used using
		following formula
		m = -(n * lg(p)) / (lg(2)^2)
		n : int
			number of items expected to be stored in filter
		p : float
			False Positive probability in decimal
		'''
		m = -(n * math.log(p))/(math.log(2)**2)
		return int(m)

	@classmethod
	def get_hash_count(self, m, n):
		'''
		Return the hash function(k) to be used using
		following formula
		k = (m/n) * lg(2)

		m : int
			size of bit array
		n : int
			number of items expected to be stored in filter
		'''
		k = (m/n) * math.log(2)
		return int(k)

Function to insert and query multiple array with bloom filter method to the blockchain.

In [33]:
def create_list(array_count, element_count, fp_prob):
  '''
  Function to create multiple array that recall bloom filter method.
  '''
  array_bf = []
  a = array_count
  n = element_count
  p = fp_prob

  for i in range(a):
    BF = BloomFilter(n, p)
    array_bf.append(BF)

  return array_bf

Code **Test**

In [34]:
n = 20 # number of items to add
p = 0.05 # false positive probability

list_bf = create_list(2, n, p)

print(list_bf)

# insert
data_1 = ['one', 'two', 'three', 'four', 'five',
          'six', 'seven', 'eight', 'nine', 'ten',
          'eleven', 'twelve', 'thirteen', 'fourteen', 'fifteen',
          'sixteen', 'seventeen', 'eighteen', 'nineteen', 'twenty']

data_2 = ['satu', 'dua', 'tiga', 'empat', 'lima',
          'enam', 'tujuh', 'delapan', 'sembilan', 'sepuluh',
          'sebelas', 'duabelas', 'tigabelas', 'empatbelas', 'limabelas',
          'enambelas', 'tujuhbelas', 'delapanbelas', 'sembilanbelas', 'duapuluh']

for item in data_1:
  list_bf[0].add(item)

for item in data_2:
  list_bf[1].add(item)

"""
NOTES: Array initialised starts from 0
"""

# query
data_test = ['one1', 'satu1', 'twoo', 'duaa', 'three', 'tiga']
for element in data_test:
  isExist = False
  for i in range(len(list_bf)):
    if (list_bf[i].check(element) == True):
      isExist = True
      break
      
  if isExist:
    print("'{}' is probably present in list number: ".format(element) + str(i) )
  else:
    print("'{}' is not exist in any list.".format(element))

list_bf[1].add('e308822895456863432914fa75c81bcf38167393674141414141426739366a5776425a6c536c51456a49636d524b5645343575656e7a6c5073346730304d6d65495730727946325157585852694d514542584c5f6e43454b546b44585051386b696c56705532642d65452d536d594c75334f6f6747504f44486e7a67484d5479616a7a6950764e79756c6f6350676870595a73715a4f577050707569736d482d9edd20b51ccb22e8e3cedf9778f6597047773bd594d41f50633d7e0a474270d52db23bb44c53167b53e33100e3667fb8f7d34eb696624a59589661716e0d2f6dac2b88b37cb76f2beda9648665af1ef4f7e09bf32c86e3e5bd18d66f326bf076c73e66c5bd5d454622678d9248e0844a9f675a0335a0abdbdb6e2bf06fddfbe5')

[<__main__.BloomFilter object at 0x1153986d0>, <__main__.BloomFilter object at 0x115398520>]
'one1' is not exist in any list.
'satu1' is not exist in any list.
'twoo' is not exist in any list.
'duaa' is not exist in any list.
'three' is probably present in list number: 0
'tiga' is probably present in list number: 1


Integrate with Database Test

In [58]:
# !pip install pymysql
import pymysql

p = 0.05

cnx = pymysql.connect (
    host='localhost',
    database='bloomfilter',
    user='admin',
    password='rhTLym-rj20TgE_0')

cursor = cnx.cursor()

cursor.execute("SELECT COUNT(*) FROM (SELECT DISTINCT block_id FROM table2) AS block_id_count")
block_count = cursor.fetchone()

# print(block_count)

list_BF = create_list(block_count[0], 6, p)

# print(list_BF)

cursor.execute("SELECT * FROM table2")
block_content = cursor.fetchall()

for bc in block_content:
    block_id = int(bc[1])
    data = bc[2]

    list_BF[block_id].add(data)

cek = 0
for bc in block_content:
    block_id = int(bc[1])
    data = bc[2]
    cek_data = list_BF[block_id].check('e308822895456863432914fa75c81bcf3816739367414141414142675f774866434b4f722d755a2d4c45637133722d6f314545637a6f6f765f325265385075474666683054486c4a54736a4649705259684f4677534d6d736b3449463648387268784c74394b66415355734a545864706f36644c56384135777755676476395050575243537474484f6c3874636e44794b36384b42665a327571427038557339b36cf62812612780a0742c2e98230a47e744f5bc5e896f3cc4133db4f31840795b116fffe71bb6fb48d33702b34c617275a8e92180a00e3a1baa0ddf71efd0a190d85ffa3d6f71701d21732584db788f5db2f0985aa66ce130776a894ab99acc512f00f001a3b92f502f2136064203d14c0cc8652238e087bd6d7400da01cc69')
    cek += 1

print(cek)
print(len(list_BF))
print(list_BF)


cnx.close()

600
100
[<__main__.BloomFilter object at 0x1154a23a0>, <__main__.BloomFilter object at 0x11538bd90>, <__main__.BloomFilter object at 0x1156e4310>, <__main__.BloomFilter object at 0x1157a0880>, <__main__.BloomFilter object at 0x1156a7970>, <__main__.BloomFilter object at 0x1156a7910>, <__main__.BloomFilter object at 0x1153aa220>, <__main__.BloomFilter object at 0x1158bba90>, <__main__.BloomFilter object at 0x1158bb3d0>, <__main__.BloomFilter object at 0x11578b580>, <__main__.BloomFilter object at 0x11578b040>, <__main__.BloomFilter object at 0x11578b8b0>, <__main__.BloomFilter object at 0x115aa2a00>, <__main__.BloomFilter object at 0x115aa22b0>, <__main__.BloomFilter object at 0x115aa27f0>, <__main__.BloomFilter object at 0x115aa2cd0>, <__main__.BloomFilter object at 0x115aa25e0>, <__main__.BloomFilter object at 0x115aa2d30>, <__main__.BloomFilter object at 0x115aa2be0>, <__main__.BloomFilter object at 0x115aa2d90>, <__main__.BloomFilter object at 0x115aa2b50>, <__main__.BloomFilter obj